## Import Dependencies

In [1]:
# importing modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os


## Read athlete height dataset

In [2]:
# athlete events df from csv
athlete_events_file = os.path.join("..","data", "rawData","athlete_events.csv")
athlete_events_df = pd.read_csv(athlete_events_file, encoding="ISO-8859-1")
athlete_events_df.head()
# len(athlete_events_df)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## Removing winter olympics

In [63]:
summer_df = athlete_events_df.loc[athlete_events_df['Season']=='Summer']

# summer_imp_df = summer_df['Height']/2.54

# converting metric to the correct system
summer_df['Height'] = (summer_df['Height']/2.54).round(decimals=2)
summer_df['Weight'] = (summer_df['Weight']/.453592).round(decimals=2)

summer_df

C:\Users\mdowl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mdowl\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,70.87,176.37,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,66.93,132.28,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,66.14,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN
27,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,66.14,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 4 x 100 metres Relay,NaN
29,10,"Einar Ferdinand ""Einari"" Aalto",M,26.0,NaN,NaN,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Swimming,Swimming Men's 400 metres Freestyle,NaN
31,12,Jyri Tapani Aalto,M,31.0,67.72,154.32,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,NaN
32,13,Minna Maarit Aalto,F,30.0,62.60,122.36,Finland,FIN,1996 Summer,1996,Summer,Atlanta,Sailing,Sailing Women's Windsurfer,NaN
33,13,Minna Maarit Aalto,F,34.0,62.60,122.36,Finland,FIN,2000 Summer,2000,Summer,Sydney,Sailing,Sailing Women's Windsurfer,NaN


## Find rows where athlete height are missing

In [64]:
# looking at records missing height
missing_df = summer_df.loc[summer_df['Height'].isnull()]
# missing_df.head()

len(missing_df)

51857

## verifying missing height row count

In [65]:
null_height_group = missing_df.groupby('Year')['ID'].count()

null_height_sum = null_height_group.sum()

null_height_sum



51857

## get full count by year to compare to missing heights

In [66]:
year_count = summer_df.groupby('Year')['ID'].count()
# year_count

missing_percent = 100* (null_height_group / year_count)

missing_percent

Year
1896    87.894737
1900    94.008264
1904    83.627978
1906    85.170225
1908    84.682361
1912    82.153465
1920    82.129543
1924    83.088095
1928    82.912660
1932    65.746042
1936    83.768829
1948    83.559719
1952    75.356711
1956    54.320265
1960     4.692696
1964     3.531550
1968     1.106195
1972     2.775621
1976     4.143039
1980     3.198442
1984     4.463719
1988     2.641854
1992    19.449796
1996    13.577649
2000     0.889950
2004     0.267797
2008     1.110131
2012     1.300310
2016     1.285798
Name: ID, dtype: float64

In [67]:
summer_clean = summer_df.loc[summer_df['Year']>=1960]

# len(summer_clean) 166,267
df = summer_clean.dropna(how='any',subset=['Height'])
df.head()


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,70.87,176.37,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,66.93,132.28,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
31,12,Jyri Tapani Aalto,M,31.0,67.72,154.32,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,NaN
32,13,Minna Maarit Aalto,F,30.0,62.60,122.36,Finland,FIN,1996 Summer,1996,Summer,Atlanta,Sailing,Sailing Women's Windsurfer,NaN
33,13,Minna Maarit Aalto,F,34.0,62.60,122.36,Finland,FIN,2000 Summer,2000,Summer,Sydney,Sailing,Sailing Women's Windsurfer,NaN


## look at sports by year (distribution)

In [70]:
# sports = df['Sport'].unique().tolist()

# sports
sportsByYear = df.groupby(['Sport','Year'])['Year'].count()

df2 = pd.DataFrame(sportsByYear)

group2 = df2.groupby('Sport')['Year'].count().reset_index()

sportList = pd.DataFrame(group2)
sportList.head()

keepSports = sportList.loc[sportList['Year']==15]

# keepSports.head()

keepSportsList = keepSports['Sport']

# type(keepSportsList)

reallyCleanData = pd.merge(keepSportsList, df, on='Sport', how='inner')


reallyCleanData.loc[reallyCleanData.Medal.isnull(), 'MedalYN']='No'
reallyCleanData.loc[reallyCleanData.Medal.notnull(), 'MedalYN']='Yes'

reallyCleanData.head(50)
# gb2 = sportsByYear.groupby('Year')['Sport'].count()
# gb2


,Sport,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Event,Medal,MedalYN
0,Athletics,18,Timo Antero Aaltonen,M,31.0,74.41,286.60,Finland,FIN,2000 Summer,2000,Summer,Sydney,Athletics Men's Shot Put,NaN,No
1,Athletics,34,Jamale (Djamel-) Aarrass (Ahrass-),M,30.0,73.62,167.55,France,FRA,2012 Summer,2012,Summer,London,"Athletics Men's 1,500 metres",NaN,No
2,Athletics,49,Moonika Aava,F,24.0,66.14,143.30,Estonia,EST,2004 Summer,2004,Summer,Athina,Athletics Women's Javelin Throw,NaN,No
3,Athletics,49,Moonika Aava,F,28.0,66.14,143.30,Estonia,EST,2008 Summer,2008,Summer,Beijing,Athletics Women's Javelin Throw,NaN,No
4,Athletics,52,Patrick Abada,M,22.0,74.41,176.37,France,FRA,1976 Summer,1976,Summer,Montreal,Athletics Men's Pole Vault,NaN,No
5,Athletics,55,Antonio Abadia Beci,M,26.0,66.93,143.30,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,"Athletics Men's 5,000 metres",NaN,No
6,Athletics,67,Mariya Vasilyevna Abakumova (-Tarabina),F,22.0,70.47,176.37,Russia,RUS,2008 Summer,2008,Summer,Beijing,Athletics Women's Javelin Throw,Silver,Yes
7,Athletics,67,Mariya Vasilyevna Abakumova (-Tarabina),F,26.0,70.47,176.37,Russia,RUS,2012 Summer,2012,Summer,London,Athletics Women's Javelin Throw,NaN,No
8,Athletics,86,Jos Manuel Abascal Gmez,M,22.0,71.65,147.71,Spain,ESP,1980 Summer,1980,Summer,Moskva,"Athletics Men's 1,500 metres",NaN,No
9,Athletics,86,Jos Manuel Abascal Gmez,M,26.0,71.65,147.71,Spain,ESP,1984 Summer,1984,Summer,Los Angeles,"Athletics Men's 1,500 metres",Bronze,Yes


## splitting out the data sets by gender

In [73]:
mens_data = reallyCleanData.loc[reallyCleanData['Sex']=='M']
# len(mens_data)

womens_data = reallyCleanData.loc[reallyCleanData['Sex']=='F']
len(womens_data)

42278

In [7]:
# noc regions
regions_file = os.path.join("..", "data", "rawData", "noc_regions.csv")
regions_df = pd.read_csv(regions_file, encoding="ISO-8859-1")
regions_df.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
